In [ ]:
targetDirectory = ""

In [1]:
# Use Case:
# papermill "DirectoryScanner.ipynb" "output.ipynb" -p "targetDirectory" "DIRECTORY_PATH"
import sys
import os
import os.path
import progressbar

class ProgressBar:
    def __init__(self):
        self.barLength = 10 # Modify this to change the length of the progress bar
        self.status = ""

    # update_progress() : Displays or updates a console progress bar
    ## Accepts a float between 0 and 1. Any int will be converted to a float.
    ## A value under 0 represents a 'halt'.
    ## A value at 1 or bigger represents 100%
    def update_progress(self, progress):      
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            self.status = "error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            self.status = "Halt...\r\n"
        if progress >= 1:
            progress = 1
            self.status = "Done...\r\n"
        block = int(round(self.barLength*progress))
        text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "-"*(self.barLength-block), str(round(progress*100, 2)), self.status)
        sys.stdout.write(text)
        sys.stdout.flush()
    
# Script intended to iterate over the sandbox looking for projects which aren't in a solution.
# Start searching
print ('Searching for projects that cannot be found in solutions:')
progress = progressbar.ProgressBar()
progress.update_progress(0) # Start a progress bar

projectCount = 0 
projectFiles = []
solutionFiles = []
solutionlessProjectFiles = []
sandboxPath = os.path.expandvars(targetDirectory)
exclude = set(['node_modules', 'packages', 'bin', 'obj'])
for dirpath, dirnames, filenames in os.walk(sandboxPath, topdown=True):
    [dirnames.remove(d) for d in list(dirnames) if d in exclude]
    for projectName in [f for f in filenames if f.endswith("proj")]:
        if projectName:
            projectFiles.append(projectName)
    for solutionName in [f for f in filenames if f.endswith(".sln")]:
        if solutionName:
            solutionFiles.append(os.path.join(dirpath, solutionName))

for projectName in projectFiles:
    projectCount = projectCount + 1
    progress.update_progress(projectCount / len(projectFiles)) # update progress bar
    found = False
    foundInFile = ''
    for solution in solutionFiles:
        if projectName in open(solution).read(): # Does the solution contain the project name?
            found = True
            foundInFile = solutionName
            break
    if(found == False):
        solutionlessProjectFiles.append(projectName) # We couldn't find a solution. Append name to the solutionless project list 

print('Solutions in directory: %s', len(solutionFiles))
print('Projects in directory: %s', len(projectFiles))
print('Solutionless Projects in directory: %s', len(solutionlessProjectFiles))
for solutionlessProject in solutionlessProjectFiles:
    print(solutionlessProject)

Searching for projects that cannot be found in solutions:
Percent: [----------] 0.0% Solutions in directory: %s 0
Projects in directory: %s 0
Solutionless Projects in directory: %s 0
